# Objetos na Memória (parte 4)

Este notebook contém exemplos do livro [_Fluent Python, Second Edition_](https://www.fluentpython.com/).

## Parâmetros de função são apelidos

Quando invocamos uma função,
os argumentos da chamada são atribuídos aos parâmetros da função.
Dentro da função, os parâmetros são apelidos (*aliases*)
dos argumentos passados.

Por esse motivo, a função pode modificar qualquer argumento mutável.
Isso é semelhante ao comportamento de algumas outras linguagens
orientadas a objetos, como JavaScript, Ruby e Java (para tipos de referência).

O exemplo abaixo mostra como uma mesma função afeta números,
listas e tuplas de maneiras diferentes. 

## Uma função pode mudar qualquer objeto mutável que receba

O exemplo abaixo mostra como a mesma função afeta números, listas e tuplas de maneiras diferentes. Isso é semelhante ao comportamento de algumas linguagens orientadas a objetos, como Javascript, Ruby e Java (para tipos de referência).

In [2]:
def f(a, b):
    a += b  # a = a + b OU a.extend(b)
    return a

x = 1
y = 2
f(x, y)

3

In [3]:
x, y

(1, 2)

In [4]:
l1 = [1, 2]
l2 = [3, 4]
f(l1, l2)

[1, 2, 3, 4]

In [5]:
l1, l2

([1, 2, 3, 4], [3, 4])

In [ ]:
t = (10, 20)
u = (30, 40)
f(t, u)

In [ ]:
t, u

## Tipos Mutáveis em Parâmetros Default: Péssima Ideia

Parâmetros opcionais com valores default são um grande recurso para definição de funções em Python, permitindo que nossas APIs evoluam mantendo a compatibilidade com versões anteriores. Entretanto, evite usar objetos mutáveis como valores default em parâmetros.

Para ilustrar esse ponto, no próximo exemplo modificamos o método `__init__` da classe `Bus` do vídeo anterior para criar `HauntedBus`, o ônibus mal-assombrado. Tentamos ser espertos, e em vez do valor default `passengers=None`, temos `passengers=[]`, a evitar o `if` do `__init__` anterior. Essa "esperteza" cria problemas.

In [8]:
class HauntedBus:
    """A bus model haunted by ghost passengers"""

    def __init__(self, passengers=[]):  # (1)
        self.passengers = passengers    # (2)

    def pick(self, name):
        self.passengers.append(name)    # (3)

    def drop(self, name):
        self.passengers.remove(name)

Veja a seguir o comportamento sobrenatural de um `HauntedBus`:

In [9]:
>>> bus1 = HauntedBus(['Alice', 'Bill'])  #(1)
>>> bus1.passengers
['Alice', 'Bill']
>>> bus1.pick('Charlie')
>>> bus1.drop('Alice')
>>> bus1.passengers  #(2)
['Bill', 'Charlie']
>>> bus2 = HauntedBus()  #(3)
>>> bus2.pick('Carrie')
>>> bus2.passengers
['Carrie']
>>> bus3 = HauntedBus()  #(4)
>>> bus3.passengers  #(5)
['Carrie']
>>> bus3.pick('Dave')
>>> bus2.passengers  #(6)
['Carrie', 'Dave']
>>> bus2.passengers is bus3.passengers  #(7)
True
>>> bus1.passengers  #(8)
['Bill', 'Charlie']

['Bill', 'Charlie']

Após executar o exemplo acima, você pode inspecionar o objeto `HauntedBu.__init__` e ver fantasmas de estudantes assombrando o atributo `__defaults__`:

In [10]:
HauntedBus.__init__.__defaults__

(['Carrie', 'Dave'],)

Veja a provar de que `bus2.passengers` e `bus3.passengers` são aliases (apelidos) para o atributo `__defaults__` do método `__init__`:

In [11]:
default_passengers = HauntedBus.__init__.__defaults__[0]

In [12]:
bus2.passengers is default_passengers

True

In [13]:
bus3.passengers is default_passengers

True

## Programação Defensiva com Parâmetros Mutáveis

Ao lidar com parâmetros mutáveis em uma função, considere se o cliente que chama a função espera que o argumento passado seja modificado.

Se a função precisa modificar o argumento, é importante decidir se esse efeito colateral deve ser visível fora da função ou não, alinhando as expectativas entre o autor da função e o cliente.

Veja `TwilightBus`, classe batizada em homenagem ao seriado sobre fenômenos paranormais "Twilight Zone".

In [15]:
class TwilightBus:
    """A bus model that makes passengers vanish"""

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []  # (1)
        else:
            self.passengers = passengers  #(2)

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)  # (3)

O comportamento paranormal:

In [16]:
>>> basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']  #(1)
>>> bus = TwilightBus(basketball_team)  # (2)
>>> bus.drop('Tina')  # (3)
>>> bus.drop('Pat')
>>> basketball_team  # (4)

['Sue', 'Maya', 'Diana']

Note que `'Tina'` e `'Pat'` desapareceram da lista `basketball_team`!

Para evitar o sumiço de pessoas do time,
uma solução simples é copiar a lista de passageiros passada como argumento,
em vez de criar um alias para ela.

In [17]:
class SafeBus:
    """A bus that doesn't make passengers vanish"""

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            # copiar para preservar o valor do argumento `passengers`
            self.passengers = list(passengers)

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [19]:
>>> basketball_team = ['Sue', 'Tina', 'Maya', 'Diana', 'Pat']  #(1)
>>> bus = SafeBus(basketball_team)  # (2)
>>> bus.drop('Tina')  # (3)
>>> bus.drop('Pat')
>>> basketball_team  # (4)

['Sue', 'Tina', 'Maya', 'Diana', 'Pat']

In [20]:
bus.passengers

['Sue', 'Maya', 'Diana']

Como vantagem adicional, `SafeBus` é mais flexível:
agora o argumento passado no parâmetro `passengers` pode ser
`tuple` ou qualquer outro tipo iterável,
como `set` ou mesmo resultados de uma consulta a um banco de dados,
pois o construtor de `list` aceita qualquer iterável.

Ao criar nossa própria lista, estamos também assegurando que
`self.passengers` suporta os métodos necessários, `.remove()` and `.append()`, que usamos nos métodos `.pick()` e `.drop()`.